In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.datasets import fashion_mnist 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['train.csv', 'test.csv']


In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
print(train.head())
print(test.head())

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0         

In [4]:
X_train = (train.iloc[:,1:]).as_matrix()
print(X_train.shape)
Y_train = train.iloc[:,0].as_matrix()
print(Y_train.shape)
X_test = test.iloc[:,:].as_matrix()
print(X_test.shape)

(42000, 784)
(42000,)
(28000, 784)


In [5]:
print(X_train.shape[0])
img_rows, img_cols = 28, 28
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

Y_train = to_categorical((Y_train))
print((Y_train.shape))

42000
(42000, 10)


In [6]:
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)
def create_model():
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=0.001))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(2500, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model

In [7]:
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          32        
__________

In [8]:
def train():
    model = create_model()
    model.compile(optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, Y_train, batch_size=64, epochs=50, verbose=1,shuffle=False, validation_split=0.3)
#     model.save_weights('./model/mnist.h5')
train()

Train on 29399 samples, validate on 12601 samples
Epoch 1/50
29399/29399 [==============================] - 18s 604us/step - loss: 0.9565 - acc: 0.6992 - val_loss: 0.1958 - val_acc: 0.9395
Epoch 2/50
29399/29399 [==============================] - 13s 455us/step - loss: 0.2778 - acc: 0.9140 - val_loss: 0.1040 - val_acc: 0.9666
Epoch 3/50
29399/29399 [==============================] - 13s 441us/step - loss: 0.1711 - acc: 0.9465 - val_loss: 0.0697 - val_acc: 0.9782
Epoch 4/50
29399/29399 [==============================] - 13s 453us/step - loss: 0.1275 - acc: 0.9608 - val_loss: 0.0564 - val_acc: 0.9821
Epoch 5/50
29399/29399 [==============================] - 14s 461us/step - loss: 0.1028 - acc: 0.9682 - val_loss: 0.0496 - val_acc: 0.9849
Epoch 6/50
29399/29399 [==============================] - 13s 458us/step - loss: 0.0886 - acc: 0.9722 - val_loss: 0.0433 - val_acc: 0.9868
Epoch 7/50
29399/29399 [==============================] - 13s 455us/step - loss: 0.0787 - acc: 0.9761 - val_loss: 0.

In [9]:
model = create_model()


In [20]:
preds = model.predict(X_test, verbose=0)[0]
print(np.argmax(preds))

2


In [ ]:
img_name=[]
label = []

for i in range(0,len(X_test)):
    print(i)
    preds = model.predict(X_test, verbose=0)[i]
    img_name.append(i)
    label.append(np.argmax(preds))
imageID = pd.DataFrame({'ImageId':img_name})
LABEL = pd.DataFrame({'Label':label})

0


2
